# Modelo de Identificação de Fraudes

In [1]:
# Importando o pandas

import pandas as pd

In [2]:
# Importando a base

transacoes = pd.read_csv('creditcard.csv')

### Criando um modelo de aprendizado de máquinas para identificar fraude
- Vamos criar um modelo para identificar fraude sem fazer nenhum tratamento nessa base

**Podemos ajustar as colunas Time e Amount para que elas fiquem entre 0 e 1**

In [3]:
# Primeiro para a coluna Time

transacoes.Time = transacoes.Time / transacoes.Time.max()

In [4]:
# E então para a coluna Amount

transacoes.Amount = transacoes.Amount / transacoes.Amount.max()

In [5]:
# Separando X e y

X = transacoes.drop('Class',axis=1)
y = transacoes.Class

**Separando em treino e teste**

In [6]:
# Separando em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### Realizando o random undersampling

In [7]:
# Importando o RandomUnderSampler do imblearn

from imblearn.under_sampling import RandomUnderSampler 

In [8]:
# Definindo o RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

In [9]:
# Definindo a nova amostra

X_resRUS, y_resRUS = rus.fit_resample(X_train, y_train)

**<font color='blue'>Com Regressão Logística**

In [10]:
# Fazendo o fit

from sklearn.linear_model import LogisticRegression

clfLog_RUS = LogisticRegression(random_state=42).fit(X_resRUS, y_resRUS)

In [11]:
# Fazendo a previsão

y_predLog_RUS = clfLog_RUS.predict(X_test)

y_pred_probaLog_RUS = clfLog_RUS.predict_proba(X_test)[:,1]

In [12]:
# Visualizando a matriz de confusão

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_predLog_RUS)

array([[91311,  2514],
       [   15,   147]], dtype=int64)

In [13]:
# Traçando a área sobre a curva precisão x recall

from sklearn import metrics

precisionLog_RUS,recallLog_RUS,thresholdsLog_RUS = metrics.precision_recall_curve(y_test,y_pred_probaLog_RUS)
print(metrics.auc(recallLog_RUS, precisionLog_RUS))

0.6949903066783492


In [14]:
# Calculando o recall

from sklearn.metrics import recall_score

recall_score(y_test, y_predLog_RUS)

0.9074074074074074

## Vamos criar um novo classificador mudando parâmetros

In [15]:
# Fazendo o novo fit

from sklearn.linear_model import LogisticRegression

clfLog_RUS2 = LogisticRegression(random_state=42, solver='newton-cg', C=100).fit(X_resRUS, y_resRUS)

In [16]:
# Fazendo a nova previsão

y_predLog_RUS2 = clfLog_RUS2.predict(X_test)

y_pred_probaLog_RUS2 = clfLog_RUS2.predict_proba(X_test)[:,1]

In [17]:
# Visualizando a nova matriz de confusão

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_predLog_RUS2)

array([[90223,  3602],
       [   14,   148]], dtype=int64)

In [18]:
# Traçando a nova área sobre a curva precisão x recall

from sklearn import metrics

precisionLog_RUS2,recallLog_RUS2,thresholdsLog_RUS2 = metrics.precision_recall_curve(y_test,y_pred_probaLog_RUS2)
print(metrics.auc(recallLog_RUS2, precisionLog_RUS2))

0.7414892735162674


In [19]:
# Calculando o novo recall

from sklearn.metrics import recall_score

recall_score(y_test, y_predLog_RUS2)

0.9135802469135802

**Podemos então [voltar](#regressao) começar a ajustar alguns parâmetros**

In [21]:
# Retirando temporariamente os warnings do nosso código

import warnings

warnings.filterwarnings('ignore')

**Vamos utilizar o GridSearchCV para automatizar a busca por melhores parâmetros**

In [48]:
# Importando o GridSearchCV

from sklearn.model_selection import GridSearchCV

In [59]:
# Definindo os parâmetros que queremos testar

parametros = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

In [70]:
# Selecionando a Regressão Logística

LogReg = LogisticRegression(random_state=42)

In [71]:
# Criando um novo classificador usando os parâmetros que escolhemos anteriormente

clf_GS = GridSearchCV(LogReg, parametros, scoring='recall')

In [72]:
# Fazendo o fit dos nossos dados

clf_GS = clf_GS.fit(X_resRUS, y_resRUS)

In [73]:
# Visualizando os melhores parâmetros definidos pelo GridSearchCV 

c.best_params_

{'C': 0.001, 'solver': 'liblinear'}

In [74]:
# Usando esse modelo para fazer as previsões

y_pred_GS = clf_GS.predict(X_test)

In [75]:
# Analisando a matriz de confusão

confusion_matrix(y_test, y_pred_GS)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [76]:
# O recall

recall_score(y_test, y_pred_GS)

0.9382716049382716

In [78]:
# E a precisão

from sklearn.metrics import precision_score

precision_score(y_test, y_pred_GS)

0.017351598173515982

In [80]:
# Podemos visualizar tudo que foi feito

pd.DataFrame(clf_GS.cv_results_)

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.022985  6.028694e-03         0.005596    4.899603e-04   0.001   
1        0.011393  1.018758e-03         0.005996    6.325608e-04   0.001   
2        0.006995  3.989506e-07         0.004997    3.234067e-07   0.001   
3        0.041574  2.057296e-03         0.005397    7.996083e-04   0.001   
4        0.054366  2.496139e-03         0.004797    9.799978e-04   0.001   
5        0.021387  2.058245e-03         0.005596    4.903496e-04    0.01   
6        0.017190  4.788450e-03         0.007595    1.623728e-03    0.01   
7        0.008194  1.468679e-03         0.005597    4.897848e-04    0.01   
8        0.040774  5.596229e-03         0.005797    3.998288e-04    0.01   
9        0.050368  4.027670e-03         0.004798    7.486728e-04    0.01   
10       0.027583  1.852944e-03         0.005996    6.326364e-04     0.1   
11       0.014991  8.938293e-04         0.005996    6.321082e-04     0.1   
12       0.008794  1.165877e-03         0.005796    3.993757e-04     0.1   
13       0.037776  3.597986e-03         0.007596    5.311906e-03     0.1   
14       0.079348  3.462427e-02         0.012393    8.793544e-03     0.1   
15       0.034778  2.225780e-03         0.005596    4.898238e-04       1   
16       0.022587  1.019487e-03         0.005396    4.900377e-04       1   
17       0.009194  3.997328e-04         0.005397    4.896095e-04       1   
18       0.035578  5.915736e-03         0.005796    3.996138e-04       1   
19       0.048568  4.713698e-03         0.004199    4.006389e-04       1   
20       0.047571  9.618536e-03         0.005796    1.166294e-03      10   
21       0.035978  6.323346e-04         0.005197    3.997326e-04      10   
22       0.012391  1.018748e-03         0.005997    6.327115e-04      10   
23       0.037576  7.083535e-03         0.004797    9.788684e-04      10   
24       0.050768  3.541777e-03         0.004598    1.019309e-03      10   
25       0.056765  5.265370e-03         0.005796    7.488764e-04     100   
26       0.041175  1.469406e-03         0.005596    4.900574e-04     100   
27       0.013391  7.992269e-04         0.005997    1.907349e-07     100   
28       0.039974  5.618495e-03         0.004998    8.935629e-04     100   
29       0.051368  4.315161e-03         0.004198    9.794427e-04     100   
30       0.081949  4.976994e-03         0.005397    4.894538e-04    1000   
31       0.040576  1.742843e-03         0.005196    3.988267e-04    1000   
32       0.021187  9.656445e-03         0.005197    1.165403e-03    1000   
33       0.034978  5.173365e-03         0.005197    1.164716e-03    1000   
34       0.046370  4.268177e-03         0.004399    4.898824e-04    1000   

   param_solver                               params  split0_test_score  \
0     newton-cg  {'C': 0.001, 'solver': 'newton-cg'}           0.757576   
1         lbfgs      {'C': 0.001, 'solver': 'lbfgs'}           0.757576   
2     liblinear  {'C': 0.001, 'solver': 'liblinear'}           0.909091   
3           sag        {'C': 0.001, 'solver': 'sag'}           0.757576   
4          saga       {'C': 0.001, 'solver': 'saga'}           0.772727   
5     newton-cg   {'C': 0.01, 'solver': 'newton-cg'}           0.833333   
6         lbfgs       {'C': 0.01, 'solver': 'lbfgs'}           0.833333   
7     liblinear   {'C': 0.01, 'solver': 'liblinear'}           0.863636   
8           sag         {'C': 0.01, 'solver': 'sag'}           0.833333   
9          saga        {'C': 0.01, 'solver': 'saga'}           0.833333   
10    newton-cg    {'C': 0.1, 'solver': 'newton-cg'}           0.863636   
11        lbfgs        {'C': 0.1, 'solver': 'lbfgs'}           0.863636   
12    liblinear    {'C': 0.1, 'solver': 'liblinear'}           0.863636   
13          sag          {'C': 0.1, 'solver': 'sag'}           0.863636   
14         saga         {'C': 0.1, 'solver': 'saga'}           0.863636   
15    newton-cg      {'C': 1, 'solver': 'newton-cg'}           0.86